# Affiliation: Whiting School of Engineering, Johns Hopkins University (685.621) Programming Assignment 2 - Problem 3



Authors: Jack Shu, Sriharshareddy Katpally, Sarah Henry


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_log_error

from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
# readh in datasets:
data_dir = '/kaggle/input/novel-corona-virus-2019-dataset'
confirmed = pd.read_csv(os.path.join(data_dir, 'time_series_covid_19_confirmed.csv'))
deaths = pd.read_csv(os.path.join(data_dir, 'time_series_covid_19_deaths.csv'))
recovered = pd.read_csv(os.path.join(data_dir, 'time_series_covid_19_recovered.csv'))

In [ ]:
print('confirmed data shape: ' + str(confirmed.shape) )
print('deaths data shape: ' + str(deaths.shape))
print('recovered data shape: ' + str(recovered.shape))

In [ ]:
# print(confirmed.columns == deaths.columns)
# print(confirmed.columns == recovered.columns)

In [ ]:
confirmed_melt = pd.melt(confirmed, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
                   var_name='Date', value_name='Confirmed')
deaths_melt = pd.melt(deaths, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
                     var_name='Date', value_name='Deaths')
recovered_melt = pd.melt(recovered, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
                        var_name='Date', value_name='Recovered')

In [ ]:
print(confirmed_melt.shape)
print(deaths_melt.shape)
print(recovered_melt.shape)

In [ ]:
df =confirmed_melt.merge(deaths_melt, how='outer', on=['Province/State', 'Country/Region', 'Lat', 'Long', 'Date'])
df = df.merge(recovered_melt, how='left', on=['Province/State', 'Country/Region', 'Lat', 'Long', 'Date'])

In [ ]:
print('the following country does not have recovery info: ' + np.unique(df[df['Recovered'].isna()]['Country/Region']))

In [ ]:
# since the number of country without recovery is relatively small, just drop them
df = df[df['Recovered'].isna() != True]

# problem 3 - 1

visualizing the cases by lat, long

In [ ]:
df_melt = pd.melt(df, id_vars = ['Province/State', 'Country/Region', 'Lat', 'Long', 'Date'])
fig = px.scatter_geo(df_melt, lat='Lat', lon='Long', size='value', color='variable',
                    animation_frame='Date', projection='natural earth')
fig.show()

# problem 3 - 2

In [ ]:
# convert to weekly average 
df_weekly = df
df_weekly.index = pd.to_datetime(df_weekly['Date'])
df_weekly = df_weekly.drop(['Province/State', 'Date', 'Lat', 'Long'], axis=1)
df_weekly = df_weekly.groupby(['Country/Region']).resample('W').mean()
df_weekly = df_weekly.reset_index()

# adding 1 to the confirmed cases to get around divide by 0 error before the spread
df_weekly['Death_Rate'] = df_weekly['Deaths'] / (df_weekly['Confirmed'] + 1)

df_weekly['Date'] = df_weekly['Date'].dt.strftime('%Y-%m-%d')

In [ ]:
random_sample_country = pd.Series(np.unique(df_weekly['Country/Region'])).sample(10, random_state=5)

# always have US
random_sample_country = random_sample_country.append(pd.Series(['US']))

df_weekly = df_weekly[df_weekly['Country/Region'].isin(random_sample_country)]
fig = px.scatter(df_weekly, x='Confirmed', y='Death_Rate', 
                 color='Country/Region',
                 range_x=[min(df_weekly['Confirmed']), max(df_weekly['Confirmed'])],
                 range_y=[min(df_weekly['Death_Rate']), max(df_weekly['Death_Rate'])],
                 animation_group='Country/Region', animation_frame='Date')

fig.update_traces(marker_size=20, opacity = 0.7)
fig.show()

# problem 3-3

Since a different visualization were applied for part 2 (e.g. bubble chart) than in problem 2, a qualitative analysis will be applied.

Comparing problem 3-1 (visualization of lat, long by date) vs. problem 2 - 1 (visualization of lat, long, by date) showed a similar pattern, the coronavirus started in Asia and then spreads to Europe and then eventually to the Americas as time progresses.

In problem 3-2, the bubble chart indicated the US number of cases initially were low (before it spread over from Aisa), followed by a rapid increase in the death rate (peaked during the week of 3/8/20). This may be due to the initial cases being more severe and hence easily detectable. As the number of cases increased, the death rate dropped off and stablized. But the number of confirmed cases increased rapidly after week of 3/15/20.

The rapid increase in the confirmed cases were consistent with the observations and predictions from problem 2-2, 2-4, 2-5 where the predictions showed a rapid increase in the number of cases, albeit not quite exponentially as one might expect for a pandemic.

From the analysis it is clear that the two factors will be critical in understanding the growth rate: time and geographical location (time & space) which will be explored in the algorithm in section problem 3-4, and 3-5


# problem 3-4

In [ ]:
df_weekly = df
df_weekly.index = pd.to_datetime(df_weekly['Date'])
df_weekly = df_weekly.groupby(['Country/Region']).resample('W').mean()
df_weekly = df_weekly.reset_index()

In [ ]:
df_knn = df_weekly

# create new feature for 'days'
df_knn['days'] = (df_knn.loc[:, 'Date'].apply(pd.Timestamp) - pd.Timestamp(df_knn.loc[:,'Date'].min())).dt.days

fig = px.scatter(df_knn, x = 'days', y = 'Country/Region', size='Confirmed', color='Country/Region')
fig.show()

df_knn = df_knn.reset_index(drop=True)

In [ ]:
# rescaling using MinMax method
X = df_knn.loc[:, ['Lat', 'Long', 'days']]
cols = X.columns
scaler = MinMaxScaler()
scaler.fit(X)
X = pd.DataFrame(scaler.transform(X), columns = cols)

# # rescaling using normalize
# X = df_svr.loc[:, ['Lat', 'Long', 'days']]
# cols = X.columns
# from sklearn.preprocessing import normalize
# X = normalize(X)

y = df_knn.loc[:, 'Confirmed']

# break into training and test data set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=15)



In [ ]:
# KNN model
neigh = KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='brute', p=2)
neigh.fit(X_train, y_train)
pred_train = neigh.predict(X_train)
pred_test = neigh.predict(X_test)

RMSLE_train = np.sqrt(mean_squared_log_error(y_train, pred_train))
RMSLE_test = np.sqrt(mean_squared_log_error(y_test, pred_test))

print('Train data RMSLE: ' + str(RMSLE_train))
print('Test data RMSLE: ' + str(RMSLE_test))
print('')

In [ ]:
# method for finding the growth rate for n-days. Defaults to start at day 0, with US coordinates

def confirmed_growth_rate(day_n, start_day=0, Lat = 37.0902, Long = -95.7129):
    day_n_X = scaler.transform(pd.DataFrame([[Lat, Long, day_n]]))
    day_n_X = pd.DataFrame(day_n_X, columns=cols)
    pred_n_cases = neigh.predict(day_n_X)
    
    start_X = scaler.transform(pd.DataFrame([[Lat, Long, start_day]]))
    start_X = pd.DataFrame(start_X, columns=cols)
    pred_start_cases = neigh.predict(start_X)
    
    # check for divide by 0 problem
    if pred_start_cases == 0:
        pred_start_cases = 1
    
    growth_rate = (pred_n_cases - pred_start_cases) / pred_start_cases
    return growth_rate



In [ ]:
day_range = np.arange(0, 30)
growth_rate = []
interval = 15 # days


for day in day_range:
    start_day = day
    day_n = day + interval
    growth_rate.append(confirmed_growth_rate(day_n, start_day)[0])

# confirmed_growth_rate(day_n=15, start_day=14)

In [ ]:
fig = px.scatter(x=day_range, y=growth_rate)
fig.show()

# problem 3-5

In [ ]:
# explore relationship between confirmed, deaths and recovered
cases = df_knn[['Confirmed', 'Deaths', 'Recovered']]
cases_melt = pd.melt(cases, id_vars='Confirmed')

fig = px.scatter(cases_melt, x='Confirmed', y='value', facet_col='variable')
fig.show()



Based on the exploratory analyis, there may be a linear (postive) relationship between the number of confirmed cases vs. the deaths and recovered, hence it may be a confounding variable. Hence, the following analyis will leave out both the 'Death' and 'Confirmed' variable


In [ ]:
# subset data to predict recovery rate
X = df_knn[['Lat', 'Long', 'days']]
cols = X.columns
scaler = MinMaxScaler()
scaler.fit(X)
X = pd.DataFrame(scaler.transform(X), columns = cols)

y = df_knn[['Recovered']]

# break into training and test data set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=15)


In [ ]:
# KNN model
neigh = KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='brute', p=2)
neigh.fit(X_train, y_train)
pred_train = neigh.predict(X_train)
pred_test = neigh.predict(X_test)

RMSLE_train = np.sqrt(mean_squared_log_error(y_train, pred_train))
RMSLE_test = np.sqrt(mean_squared_log_error(y_test, pred_test))

print('Train data RMSLE: ' + str(RMSLE_train))
print('Test data RMSLE: ' + str(RMSLE_test))
print('')

In [ ]:
# method for finding the recovery rate for n-days. Defaults to start at day 0, with US coordinates

def recovery_rate(day_n, start_day=0, Lat = 37.0902, Long = -95.7129):
    day_n_X = scaler.transform(pd.DataFrame([[Lat, Long, day_n]]))
    day_n_X = pd.DataFrame(day_n_X, columns=cols)
    pred_n_cases = neigh.predict(day_n_X)
    
    start_X = scaler.transform(pd.DataFrame([[Lat, Long, start_day]]))
    start_X = pd.DataFrame(start_X, columns=cols)
    pred_start_cases = neigh.predict(start_X)
    
    # check for divide by 0 problem
    if pred_start_cases == 0:
        pred_start_cases = 1
    
    recovery_rate = (pred_n_cases - pred_start_cases) / pred_start_cases
    return recovery_rate

In [ ]:
day_range = np.arange(0, 30)
recover_rate = []
interval = 15 # days


for day in day_range:
    start_day = day
    day_n = day + interval
    recover_rate.append(recovery_rate(day_n, start_day)[0])


# recovery_rate(day_n=15, start_day=14)

In [ ]:
fig = px.scatter(x = day_range, y= recover_rate)
fig.show()

# run time analysis for KNN (problem 3-4 & 3-5)

For traning the KNN model it just requires storing the X_train and y_train values, which only requires constant time. For predicting the KNN model the distance of every X_test would need be be compared with every X_train to calculate the distance.

Assuming there is 'd' dimensions, with 'n' X_train data points and 'm' observations in X_test, then the run time would then be T(n) = d * n * m. Which follows the form of T(n) = a * n + b, thereform the time complexity would be of O(n)
